In [61]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import pandas as pd
import numpy as np
import copy
import time
from retry import retry

In [8]:
def sound():
    !rundll32 user32.dll,MessageBeep

In [3]:
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50'
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')

In [46]:
columns = ['建物名','住所','最寄り駅1','最寄り駅2','最寄り駅3','築年数','階数','賃料','管理費','敷金','礼金','間取り','専有面積']
df = pd.DataFrame(data_mother, columns=columns)

In [124]:
mother = soup.find_all(class_='cassetteitem')
data_samples = []
for child in mother:
    
    # 建物情報
    data_home = []
    data_home.append(child.find(class_='cassetteitem_content-title').text)
    data_home.append(child.find(class_='cassetteitem_detail-col1').text)
    children = child.find(class_='cassetteitem_detail-col2')
    for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
        data_home.append(grandchild.text)
    children = child.find(class_='cassetteitem_detail-col3')
    for grandchild in children.find_all('div'):
        data_home.append(grandchild.text)
    
    # 部屋情報
    rooms = child.find(class_='cassetteitem_other')
    for room in rooms.find_all(class_='js-cassette_link'):
        data_room = []
        for id_, grandchild in enumerate(room.find_all('td')):
            if id_ == 2:
                data_room.append(grandchild.text.strip())
            elif id_ == 3:
                data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
            elif id_ == 4:
                data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
            elif id_ == 5:
                data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
        data_sample = data_home + data_room
        data_samples.append(data_sample)

In [126]:
columns = ['建物名','住所','最寄り駅1','最寄り駅2','最寄り駅3','築年数','階数','階','賃料','管理費','敷金','礼金','間取り','専有面積']
df = pd.DataFrame(data_samples, columns=columns)

In [145]:
df

,建物名,住所,最寄り駅1,最寄り駅2,最寄り駅3,築年数,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積
0,プチ・ボヌール,東京都大田区南蒲田３,京急空港線/糀谷駅 歩7分,京急本線/京急蒲田駅 歩10分,京急本線/雑色駅 歩16分,築4年,2階建,2階,14.8万円,5000円,14.8万円,14.8万円,42DK,48.87m2
1,ドリームハウス熊野前,東京都荒川区東尾久５,日暮里・舎人ライナー/熊野前駅 歩3分,ＪＲ高崎線/尾久駅 歩18分,ＪＲ山手線/田端駅 歩20分,築6年,3階建,2階,8.9万円,3000円,-,-,1LDK,33.94m2
2,東京メトロ千代田線 町屋駅 9階建 築17年,東京都荒川区町屋２,東京メトロ千代田線/町屋駅 歩6分,京成本線/新三河島駅 歩15分,日暮里・舎人ライナー/赤土小学校前駅 歩16分,築17年,9階建,8階,11.3万円,5000円,11.3万円,11.3万円,1LDK,41.04m2
3,いずみマンションII,東京都荒川区町屋２,東京メトロ千代田線/町屋駅 歩6分,都電荒川線/町屋二丁目駅 歩2分,京成本線/新三河島駅 歩15分,築17年,9階建,8階,11.3万円,5000円,11.3万円,11.3万円,1LDK,41.04m2
4,Ｓｅｐｉａ Ｐａｌａｃｅ,東京都荒川区東日暮里６,京成本線/日暮里駅 歩8分,日暮里・舎人ライナー/日暮里駅 歩7分,ＪＲ常磐線/三河島駅 歩8分,新築,3階建,1階,11.9万円,5000円,-,11.9万円,1LDK,33.53m2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,エクセルIII,東京都足立区六町２,つくばエクスプレス/六町駅 歩5分,つくばエクスプレス/青井駅 歩17分,東京メトロ千代田線/北綾瀬駅 歩23分,新築,3階建,3階,11万円,5500円,-,11万円,1LDK,38.11m2
138,エクセルIII,東京都足立区六町２,つくばエクスプレス/六町駅 歩5分,つくばエクスプレス/青井駅 歩17分,東京メトロ千代田線/北綾瀬駅 歩23分,新築,3階建,1階,12.1万円,5500円,-,12.1万円,2LDK,48.1m2
139,エクセルIII,東京都足立区六町２,つくばエクスプレス/六町駅 歩5分,つくばエクスプレス/青井駅 歩17分,東京メトロ千代田線/北綾瀬駅 歩23分,新築,3階建,1階,12.1万円,5500円,-,12.1万円,2LDK,47.83m2
140,エクセルIII,東京都足立区六町２,つくばエクスプレス/六町駅 歩5分,つくばエクスプレス/青井駅 歩17分,東京メトロ千代田線/北綾瀬駅 歩23分,新築,3階建,3階,13万円,5500円,-,13万円,2LDK,48.1m2


In [ ]:
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page=2

In [164]:
# ページシフトを作る
page = 2
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page={}'
html = requests.get(url.format(page))
soup = BeautifulSoup(html.content, 'html.parser')

In [161]:
data_samples = []

mother = soup.find_all(class_='cassetteitem')
for child in mother:
    
    # 建物情報
    data_home = []
    data_home.append(child.find(class_='cassetteitem_content-title').text)
    data_home.append(child.find(class_='cassetteitem_detail-col1').text)
    children = child.find(class_='cassetteitem_detail-col2')
    for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
        data_home.append(grandchild.text)
    children = child.find(class_='cassetteitem_detail-col3')
    for grandchild in children.find_all('div'):
        data_home.append(grandchild.text)
    
    # 部屋情報
    rooms = child.find(class_='cassetteitem_other')
    for room in rooms.find_all(class_='js-cassette_link'):
        data_room = []
        for id_, grandchild in enumerate(room.find_all('td')):
            if id_ == 2:
                data_room.append(grandchild.text.strip())
            elif id_ == 3:
                data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
            elif id_ == 4:
                data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
            elif id_ == 5:
                data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
        data_sample = data_home + data_room
        data_samples.append(data_sample)

In [165]:
columns = ['建物名','住所','最寄り駅1','最寄り駅2','最寄り駅3','築年数','階数','階','賃料','管理費','敷金','礼金','間取り','専有面積']
df = pd.DataFrame(data_samples, columns=columns)

In [168]:
df

,建物名,住所,最寄り駅1,最寄り駅2,最寄り駅3,築年数,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積
0,フロール・ガーデン,東京都荒川区東尾久４,ＪＲ山手線/田端駅 歩13分,東京メトロ千代田線/町屋駅 歩15分,日暮里・舎人ライナー/赤土小学校前駅 歩3分,築12年,3階建,3階,7.4万円,2000円,7.4万円,-,ワンルーム,33.12m2
1,ブランドール..,東京都荒川区町屋１,東京メトロ千代田線/町屋駅 歩10分,都電荒川線/町屋駅前駅 歩9分,都電荒川線/荒川七丁目駅 歩8分,築18年,3階建,2階,7.5万円,3500円,3.75万円,7.5万円,1K,26.71m2
2,ルミナス東葛西,東京都江戸川区東葛西６,東京メトロ東西線/葛西駅 歩8分,東京メトロ東西線/西葛西駅 歩24分,東京メトロ東西線/浦安駅 歩27分,築10年,4階建,1階,11.7万円,2000円,11.7万円,11.7万円,1LDK,44.12m2
3,フリービング南千住,東京都荒川区南千住２,東京メトロ日比谷線/三ノ輪駅 歩4分,ＪＲ常磐線/南千住駅 歩4分,東京メトロ日比谷線/南千住駅 歩4分,築4年,4階建,1階,6.9万円,6000円,6.9万円,6.9万円,ワンルーム,18.73m2
4,エクセルIII,東京都足立区六町２,つくばエクスプレス/六町駅 歩5分,つくばエクスプレス/青井駅 歩17分,東京メトロ千代田線/北綾瀬駅 歩23分,新築,3階建,1階,10.5万円,5500円,-,10.5万円,1LDK,39.92m2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,パークサイド・ウイングス,東京都中央区佃１,東京メトロ有楽町線/月島駅 歩6分,東京メトロ日比谷線/八丁堀駅 歩11分,ＪＲ京葉線/越中島駅 歩10分,築35年,14階建,6階,26.4万円,-,26.4万円,26.4万円,2LDK,70.37m2
213,パークサイド・ウイングス,東京都中央区佃１,東京メトロ有楽町線/月島駅 歩6分,東京メトロ日比谷線/八丁堀駅 歩11分,ＪＲ京葉線/越中島駅 歩10分,築35年,14階建,10階,27.3万円,-,27.3万円,27.3万円,2LDK,73.36m2
214,パークサイド・ウイングス,東京都中央区佃１,東京メトロ有楽町線/月島駅 歩6分,東京メトロ日比谷線/八丁堀駅 歩11分,ＪＲ京葉線/越中島駅 歩10分,築35年,14階建,1階,27.7万円,-,27.7万円,27.7万円,2LDK,71.97m2
215,パークサイド・ウイングス,東京都中央区佃１,東京メトロ有楽町線/月島駅 歩6分,東京メトロ日比谷線/八丁堀駅 歩11分,ＪＲ京葉線/越中島駅 歩10分,築35年,14階建,10階,28万円,-,28万円,28万円,2LDK,73.36m2


In [110]:
# 複数ページの情報をまとめて取得
data_samples = []

# スクレイピングするページ数
max_page = 2897
# SUUMOを東京都23区のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page={}'

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
start = time.time()
times = []

# ページごとの処理
for page in range(1,max_page+1):
    before = time.time()
    # ページ情報
    soup = load_page(url.format(page))
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')
    
    # 物件ごとの処理
    for child in mother:

        # 建物情報
        data_home = []
        # 建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        # 住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        # 最寄り駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        # 築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)

        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                # 階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                # 家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                # 敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                # 間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
            # 物件情報と部屋情報をくっつける
            data_sample = data_home + data_room
            data_samples.append(data_sample)
    
    # 1アクセスごとに1秒休む
    time.sleep(1)
    
    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)
sound()

1ページ目：6.918519735336304秒
完了：0.035%
残り時間：5時間33分56秒

2ページ目：6.420626401901245秒
完了：0.069%
残り時間：5時間21分48秒

3ページ目：7.214626789093018秒
完了：0.104%
残り時間：5時間30分27秒

4ページ目：9.227331638336182秒
完了：0.138%
残り時間：5時間58分59秒

5ページ目：12.644416809082031秒
完了：0.173%
残り時間：6時間48分58秒

6ページ目：7.748034238815308秒
完了：0.207%
残り時間：6時間42分55秒

7ページ目：9.581820011138916秒
完了：0.242%
残り時間：6時間51分10秒

8ページ目：9.654930353164673秒
完了：0.276%
残り時間：6時間57分45秒

9ページ目：8.57774019241333秒
完了：0.311%
残り時間：6時間57分5秒

10ページ目：7.974159002304077秒
完了：0.345%
残り時間：6時間53分37秒

11ページ目：8.416781187057495秒
完了：0.38%
残り時間：6時間52分41秒

12ページ目：7.553883790969849秒
完了：0.414%
残り時間：6時間48分26秒

13ページ目：7.770699739456177秒
完了：0.449%
残り時間：6時間45分37秒

14ページ目：7.99126410484314秒
完了：0.483%
残り時間：6時間43分56秒

15ページ目：9.525159358978271秒
完了：0.518%
残り時間：6時間47分23秒



KeyboardInterrupt: 

In [77]:
columns = ['建物名','住所','最寄り駅1','最寄り駅2','最寄り駅3','築年数','階数','階','賃料','管理費','敷金','礼金','間取り','専有面積']
suumo_tokyo = pd.DataFrame(data_samples, columns=columns)

In [78]:
suumo_tokyo

,建物名,住所,最寄り駅1,最寄り駅2,最寄り駅3,築年数,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積
0,プチ・ボヌール,東京都大田区南蒲田３,京急空港線/糀谷駅 歩7分,京急本線/京急蒲田駅 歩10分,京急本線/雑色駅 歩16分,築4年,2階建,2階,14.8万円,5000円,14.8万円,14.8万円,42DK,48.87m2
1,ドリームハウス熊野前,東京都荒川区東尾久５,日暮里・舎人ライナー/熊野前駅 歩3分,都電荒川線/熊野前駅 歩2分,ＪＲ山手線/田端駅 歩20分,築6年,3階建,2-3階,8.9万円,3000円,-,-,1LDK,33.94m2
2,東京メトロ千代田線 町屋駅 9階建 築17年,東京都荒川区町屋２,東京メトロ千代田線/町屋駅 歩6分,都電荒川線/町屋二丁目駅 歩3分,都電荒川線/町屋駅前駅 歩5分,築17年,9階建,8階,11.3万円,5000円,11.3万円,11.3万円,1LDK,41.04m2
3,いずみマンションII,東京都荒川区町屋２,東京メトロ千代田線/町屋駅 歩6分,京成本線/新三河島駅 歩14分,ＪＲ山手線/西日暮里駅 歩23分,築17年,9階建,8階,11.3万円,5000円,11.3万円,11.3万円,1LDK,41.04m2
4,Sepia Palace,東京都荒川区東日暮里６,ＪＲ山手線/日暮里駅 歩8分,ＪＲ山手線/西日暮里駅 歩14分,ＪＲ常磐線/三河島駅 歩8分,新築,3階建,1階,11.9万円,5000円,-,11.9万円,1LDK,33.53m2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,グランドコンシェルジュ新御徒町アジールコート,東京都台東区元浅草１,ＪＲ山手線/御徒町駅 歩10分,都営大江戸線/新御徒町駅 歩2分,東京メトロ日比谷線/仲御徒町駅 歩8分,築8年,13階建,9階,11.1万円,10500円,-,11.1万円,1K,25.15m2
2352,グランドコンシェルジュ新御徒町アジールコート,東京都台東区元浅草１,ＪＲ山手線/御徒町駅 歩10分,都営大江戸線/新御徒町駅 歩2分,東京メトロ日比谷線/仲御徒町駅 歩8分,築8年,13階建,9階,11.3万円,10500円,-,11.3万円,1K,25.15m2
2353,グランドコンシェルジュ新御徒町アジールコート,東京都台東区元浅草１,ＪＲ山手線/御徒町駅 歩10分,都営大江戸線/新御徒町駅 歩2分,東京メトロ日比谷線/仲御徒町駅 歩8分,築8年,13階建,6階,11.4万円,10500円,-,11.4万円,1K,26.42m2
2354,グランドコンシェルジュ新御徒町アジールコート,東京都台東区元浅草１,ＪＲ山手線/御徒町駅 歩10分,都営大江戸線/新御徒町駅 歩2分,東京メトロ日比谷線/仲御徒町駅 歩8分,築8年,13階建,7階,15.5万円,10000円,15.5万円,15.5万円,1LDK,40.27m2


In [79]:
# 管理費の'-'を0に置き換える
# 管理費をint型に変換する
suumo_tokyo.loc[suumo_tokyo['管理費']=="-",'管理費'] = suumo_tokyo.loc[suumo_tokyo['管理費']=="-",'管理費'].replace('-','0')
suumo_tokyo['管理費'] = suumo_tokyo['管理費'].str.replace('円','').astype(int)

In [86]:
# 新築は築年数0年にする
# 築年数をint型に変換する
suumo_tokyo.loc[suumo_tokyo['築年数']=="新築",'築年数'] = suumo_tokyo.loc[suumo_tokyo['築年数']=="新築",'築年数'].replace('新築','0')
suumo_tokyo['築年数'] = suumo_tokyo['築年数'].str.replace('築','').str.replace('年','').astype(int)

In [119]:
# 地下地上階数変数を作る
df_ suumo_tokyo[suumo_tokyo['階数'].map(lambda x: x.find('地上'))==3]

,建物名,住所,最寄り駅1,最寄り駅2,最寄り駅3,築年数,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積
84,ライジングプレイス南千住,東京都荒川区南千住２,東京メトロ日比谷線/南千住駅 歩4分,東京メトロ日比谷線/三ノ輪駅 歩10分,東武伊勢崎線/浅草駅 バス13分 (バス停)泪橋 歩2分,16,地下1地上15階建,14階,8.8万円,0,8.8万円,-,1K,25m2
85,ライジングプレイス南千住,東京都荒川区南千住２,東京メトロ日比谷線/南千住駅 歩4分,東京メトロ日比谷線/三ノ輪駅 歩10分,東武伊勢崎線/浅草駅 バス13分 (バス停)泪橋 歩2分,16,地下1地上15階建,12階,8万円,8000,8万円,-,1K,24.93m2
86,ライジングプレイス南千住,東京都荒川区南千住２,東京メトロ日比谷線/南千住駅 歩4分,東京メトロ日比谷線/三ノ輪駅 歩10分,東武伊勢崎線/浅草駅 バス13分 (バス停)泪橋 歩2分,16,地下1地上15階建,3階,8.5万円,10500,-,8.5万円,1K,25.14m2
116,サンクレール荒川ブランガーデン,東京都荒川区荒川１,東京メトロ日比谷線/三ノ輪駅 歩7分,東京メトロ千代田線/町屋駅 歩19分,ＪＲ常磐線/三河島駅 歩14分,18,地下1地上18階建,13階,8.1万円,8000,8.1万円,8.1万円,ワンルーム,27.43m2
117,サンクレール荒川ブランガーデン,東京都荒川区荒川１,東京メトロ日比谷線/三ノ輪駅 歩7分,東京メトロ千代田線/町屋駅 歩19分,ＪＲ常磐線/三河島駅 歩14分,18,地下1地上18階建,15階,8.2万円,8000,8.2万円,-,ワンルーム,27.43m2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,シンシア学芸大学,東京都目黒区鷹番１,東急東横線/学芸大学駅 歩9分,東急東横線/都立大学駅 歩17分,ＪＲ山手線/目黒駅 バス15分 (バス停)田向公園 歩1分,23,地下1地上5階建,2階,8.2万円,8000,8.2万円,8.2万円,1K,21.4m2
2321,LAPiS 恵比寿II,東京都目黒区三田２,ＪＲ山手線/恵比寿駅 歩10分,ＪＲ山手線/目黒駅 歩10分,東急東横線/代官山駅 歩18分,2,地下1地上5階建,2階,13.3万円,8000,13.3万円,-,1K,23.49m2
2322,LAPiS 恵比寿II,東京都目黒区三田２,ＪＲ山手線/恵比寿駅 歩10分,ＪＲ山手線/目黒駅 歩10分,東急東横線/代官山駅 歩18分,2,地下1地上5階建,3階,13.4万円,8000,13.4万円,-,1K,23.49m2
2323,LAPiS 恵比寿II,東京都目黒区三田２,ＪＲ山手線/恵比寿駅 歩10分,ＪＲ山手線/目黒駅 歩10分,東急東横線/代官山駅 歩18分,2,地下1地上5階建,B1-1階,21.8万円,10000,21.8万円,-,1LDK,42.68m2


In [104]:
suumo_tokyo['階数'].unique()

array(['2階建', '3階建', '9階建', '7階建', '10階建', '11階建', '14階建', '8階建', '12階建',
       '5階建', '20階建', '地下1地上15階建', '4階建', '13階建', '6階建', '地下1地上18階建',
       '地下1地上3階建', '地下1地上13階建', '地下1地上5階建', '地下1地上11階建', '15階建',
       '地下1地上10階建', '22階建', '地下2地上60階建', '地下2地上43階建', '地下1地上14階建',
       '地下1地上4階建', '地下1地上27階建', '地下4地上35階建', '地下1地上12階建', '地下2地上14階建',
       '24階建', '地下2地上31階建', '地下1地上20階建', '17階建', '地下2地上18階建', '地下1地上8階建',
       '地下3地上42階建'], dtype=object)